In [1]:
import sys
import csv

from pathlib import Path

import psycopg2

import pandas as pd

sys.path.append('../../')
from utils import get_config

In [2]:
resource_p = Path("../resources")

food_p = resource_p / "food_business.csv"

categories_p = resource_p / "complete_category_list.csv"

# hdf_p       = resource_p / "unhealthy_businesses.csv"
noise_hdf_p = resource_p / "noise_unhealthy_food_biz.csv"
noise_all_p = resource_p / "noise_all_food_biz.csv"

In [3]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")

In [9]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [10]:
# Get resgistrant attributes
qry = """
WITH unnested_cats AS (SELECT unnest(categories) AS cats
FROM ca_business)
SELECT cats, count(cats) as category_count
FROM unnested_cats
GROUP BY cats
ORDER BY category_count DESC;
"""


cur = conn.cursor()

# execute a statement
cur.execute(qry)


# display the PostgreSQL database server version
res = cur.fetchall()
# res = [c[0] for c in col_names]
       
# Close the communication with the PostgreSQL
cur.close()

In [12]:
categories_df = pd.DataFrame(res, columns=["categories", "category_count"])

# categories_df = pd.read_csv(categories_p)
# categories_df.rename(columns={"cats": "categories"}, inplace=True)
categories_df

,categories,category_count
0,Restaurant,38523
1,Fast food restaurant,15560
2,Mexican restaurant,13969
3,Auto repair shop,13163
4,Beauty salon,10758
...,...,...
4015,Regional council,1
4016,Residential area,1
4017,Sheepskin and wool products supplier,1
4018,Ladder supplier,1


In [3]:
def read_categories_csv(p):
    with open(p) as csvfile:
        reader = csv.reader(csvfile)
        res = []
        for row in reader:
            res.append(row[0])
    return res

In [4]:
# read in food related csv
hdf_noise_df = pd.read_csv(noise_hdf_p, delimiter="#", header=None)
hdf_noise_df.rename(columns={0: "categories"}, inplace=True)
hdf_noise_df.head()

,categories
0,Fast food restaurant
1,Convenience store
2,Liquor store
3,Ice cream shop
4,Pizza restaurant


In [5]:
# read in food related csv
food_df = pd.read_csv(food_p, delimiter="#", header=None)
food_df.rename(columns={0: "categories"}, inplace=True)
food_df.head()

,categories
0,Agricultural organization
1,Bagel shop
2,Butcher shop
3,Sandwich shop
4,Cookie shop


In [11]:
# https://stackoverflow.com/a/37864171
hdf_list = ["Fast food restaurant",
            "Convenience store",
            "Liquor store",
            "Ice cream shop",
            "Pizza restaurant",
            "Pizza Takeout",
            "Sports bar",
            "Brewpub",
            "Brewery",
            "Candy store",
            "Hamburger restaurant",
            "Chicken wings restaurant",
            "Sports bar",
            "Dessert shop",
            "Donut shop",
            "Cocktail bar",
            "Wine bar",
            "Wine store",
            "liquor",
            " bar",
            "bar ",
            "pizza",
            "hamburger",
            "dessert",
            "ice cream",
            "patisserie",
            "Fast food", "Soft drinks shop"
            "donut", "Beer",
            "fried", 'fries',
            'Crêperie','Pastry shop',
            'Confectionery','Hot dog restaurant', 'Hot dog stand', 'hot dog'
            'Chocolate shop','Chocolate',
            'Cookie shop', 'cookie', 'Cupcake shop', 'Pie shop', 'Popcorn Store',
            'Beer garden', 'shakes', "sweet", "Irish pub"
]
# hdf_noise_list = read_categories_csv(noise_hdf_p)

hdf_noise_list = ["eyebrow bar", "Stand bar", 'Convenience stores organization', 'Military barracks', 'Espresso bar', 'poke bar',
              'Korean barbecue restaurant','Hookah bar','Karaoke bar','Live music bar','Eyebrow bar', 'Bar restaurant furniture store',
              'Mongolian barbecue restaurant','Frozen dessert supplier', 'Bar stool supplier', 'Poke bar',
               'Ice cream equipment supplier', 'Stand bar', 'Salsa bar', 'Convenience stores organization', 'Military barracks',
              "tapas bar", "snack bar", "Wine storage facility", "Wine club", "Wine cellar", "Wine wholesaler and importer",
              "Winemaking supply store", 'Rice mill', 'Chocolate factory', 'Oyster bar restaurant', "Beer distributor"
             ]

m = (categories_df["categories"].str.contains('|'.join(hdf_list), case=False)) | (categories_df["categories"] == "Bar")
filtered_hpf = categories_df.loc[m]

# Remove some noisy hits from matching on bar
filtered_hpf = filtered_hpf[~filtered_hpf["categories"].str.contains('|'.join(hdf_noise_list), case=False)]

filtered_hpf["categories"].to_csv(resource_p / "unhealthy_businesses.csv", index=False)

filtered_hpf

,categories,category_count
1,Fast food restaurant,15560
16,Bar,8184
18,Pizza restaurant,7898
21,Hamburger restaurant,7281
22,Convenience store,7107
37,Liquor store,4372
45,Ice cream shop,3960
59,Dessert shop,3297
62,Pizza delivery,3229
79,Bar & grill,2857


In [12]:
# make total food list
all_food_list = food_df.categories.tolist() + hdf_list
all_food_list = list(set(all_food_list))
all_food_list + ['yogurt']

noise_list = ["eyebrow bar", "Stand bar", 'Convenience stores organization', 'Military barracks',
              'Hookah bar','Karaoke bar','Live music bar','Eyebrow bar', 'Bar restaurant furniture store',
              'Frozen dessert supplier', 'Bar stool supplier', 'GRAIN & OILSEED MILLING', 
               'Ice cream equipment supplier', 'Stand bar', 'Convenience stores organization', 'Military barracks',
               "Wine storage facility", "Wine club", "Wine cellar", "Wine wholesaler and importer", 'crop grower',
              "Winemaking supply store",'Food production', 'Food processing company', 'food broker', 'Agricultural organization',
              'Cooking school', 'Fish farm', 'Meat packer', 'Fish processing', 'MEAT PROCESSED FROM CARCASSES',
              'FOOD SERVICE CONTRACTORS', "Barber shop", "Internet cafe", "Beer distributor",
              "Flower delivery", "Grill store", "Barber supply store", "Farm equipment supplier",
              "Restaurant supply store", "Food manufacturer", "Christmas tree farm", "Seed supplier",
              "Farm equipment repair service", "Barber school", "Art cafe", "Chocolate factory",
              "Food manufacturing supply", "Bark supplier", "Tree farm", "Coffee machine supplier",
              "Food and beverage consultant", "Modeling agency", "Barbecue area", "Olive oil manufacturer",
              "Agriculture cooperative", "Dairy farm equipment supplier", "Alcohol manufacturer", "Frozen food manufacturer"
              "Food machinery supplier", "Dude ranch", "Food processing equipment", "Bartending school",
              "Farmstay", "Modeling school", "Condiments supplier", "Farm household tour",
              "Camping farm", "Oil wholesaler", "Clothing wholesale market place", "Farm bureau",
              "Tea manufacturer", "Cabaret club", "Food seasoning manufacturer",
              "Childrens farm", "Rice mill", "Barrister", "Barrel supplier", "Farm school",
              "Wind farm", "Shop supermarket furniture store", "Bariatric surgeon", "Truck farmer",
              "Pet food and animal feeds", "College of agriculture", "Beverage supplier",
              "Horsestable studfarm", "Food and beverage distributors", "Floating market"
             ]



m = (categories_df["categories"].str.contains('|'.join(all_food_list), case=False)) | (categories_df["categories"] == "Bar")
filtered_food = categories_df.loc[m]

# Remove some noisy hits from matching on bar
filtered_food = filtered_food[~filtered_food["categories"].str.contains('|'.join(noise_list), case=False)]

filtered_food["categories"].to_csv(resource_p / "food_businesses.csv", index=False)

filtered_food

/var/folders/wd/y5szpxf50s70ykknr5mg_1n00000gn/T/ipykernel_23503/3485648597.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  m = (categories_df["categories"].str.contains('|'.join(all_food_list), case=False)) | (categories_df["categories"] == "Bar")


,categories,category_count
0,Restaurant,38523
1,Fast food restaurant,15560
2,Mexican restaurant,13969
5,Coffee shop,10487
7,Grocery store,10070
...,...,...
3952,Industrial supermarket,1
3962,Anago restaurant,1
3982,Chanko restaurant,1
3993,Alsace restaurant,1


In [13]:
noise_df = pd.DataFrame(noise_list)
noise_df.rename(columns={0: "categories"}, inplace=True)
noise_df.to_csv(resource_p / "noise_all_food_biz.csv", index=False, header=None)

noise_df = pd.DataFrame(hdf_noise_list)
noise_df.rename(columns={0: "categories"}, inplace=True)
noise_df.to_csv(resource_p / "noise_unhealthy_food_biz.csv", index=False, header=None)